<h1>Import and install of spark</h1>

In [1]:
#### This cell is to make spark work on a windows laptop
import os
import sys

# Path for spark source folder
os.environ['SPARK_HOME']="C:\spark-2.0.1-bin-hadoop2.7"

# Append pyspark  to Python Path
sys.path.append("C:\spark-2.0.1-bin-hadoop2.7\python")
sys.path.append("C:\spark-2.0.1-bin-hadoop2.7\python\lib\py4j-0.10.3-src.zip")
#os.environ['SPARK_EXECUTOR_MEMORY']="5G"

try:
    from pyspark import SparkContext
    from pyspark import SparkConf
    from pyspark.sql import SQLContext
    print ("Successfully imported Spark Modules")

except ImportError as e:
    print ("Can not import Spark Modules", e)
    sys.exit(1)

# Initialize SparkContext
sc = SparkContext('local[*]')
words = sc.parallelize(["scala","java","hadoop","spark","akka"])
print (words.count())
print(words.countByValue())

Successfully imported Spark Modules
5
defaultdict(<class 'int'>, {'akka': 1, 'hadoop': 1, 'scala': 1, 'java': 1, 'spark': 1})


In [2]:
from pyspark.sql import types
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np

<h1>Import data and take off header</h1>

In [3]:
input_path = 'train.csv'
raw_data = sc.textFile(input_path)

print("number of rows before cleaning header:",raw_data.count())

header = raw_data.first()

cleaned_data = raw_data.filter(lambda row : row != header)

print("number of rows without header:",cleaned_data.count())
print('Number of partitions :'+str(cleaned_data.getNumPartitions()))

sqlContext = SQLContext(sc)

number of rows before cleaning header: 188319
number of rows without header: 188318
Number of partitions :2


In [4]:
names = header.split(';')[0].split(',')
print(names)
print("Length of names",len(names))

['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'cat109', 'cat110', '

In [5]:
cats = names[1:117]
conts = names[117:-1]

In [6]:
def create_StructField(string):
    hint = string[:3]
    if hint == "cat":
        datatype = types.IntegerType()
    elif hint == "con":
        datatype = types.FloatType()
    elif hint == "id":
        datatype = types.IntegerType()
    elif hint == "los":
        datatype = types.FloatType()
    else:
        raise ValueError("Can\'t read this string:" + hint )

    return types.StructField(string, datatype, False)

In [7]:
structField_list = [create_StructField(string) for string in names]

In [8]:
data_schema = types.StructType(structField_list)

In [9]:
def tryeval(val,column_number):
    if column_number == 0:
        return int(val)
    elif 1 <= column_number <= 116:
        return val
    elif 117 <= column_number <= 131:
        return float(val)
    else:
        raise Exception("There is a big problem")

def to_tuple(row):
    list_strings = row.split(',')
    return tuple(tryeval(val, n) for n, val in enumerate(list_strings))

cleaned_data_splitted = cleaned_data.map(lambda x:to_tuple(x))

    

In [10]:
def to_tuples(list_):
    return tuple((string,) for string in list_)

def fusion(x, y):
    return tuple(tuple(set(xi + yi)) for xi, yi in zip(x,y))

list_of_dictionaries = []
a = cleaned_data_splitted.map(lambda x: to_tuples(x[1:117]))
a = a.reduce(fusion)

In [11]:
sorted_tuples = tuple(tuple(sorted(tup)) for tup in a)

In [12]:
for tup in sorted_tuples:
    my_dict = dict()
    for idx, cat in enumerate(tup):
        my_dict[cat] = idx
    list_of_dictionaries.append(my_dict)

In [13]:
bListOfDictionaries = sc.broadcast(list_of_dictionaries)

In [14]:
def replace(row):
    strings = row[1:117]
    my_dicts = bListOfDictionaries.value
    tuple_of_ints = ()
    for dict_, string in zip(my_dicts, strings):
        try:
            tuple_of_ints += (dict_[string],)
        except KeyError:
            tuple_of_ints += (0.0,)
    return (row[0],) + tuple_of_ints + row[117:]

In [15]:
final_rdd = cleaned_data_splitted.map(lambda row:replace(row))

In [16]:
df = sqlContext.createDataFrame(final_rdd, schema = data_schema).coalesce(12).cache()

<h1>PART OF MAKING A PREDICTION </h1>

In [17]:
from pyspark.ml.linalg import Vectors, VectorUDT

In [18]:
def keep_index(tup):
    result = ()
    for idx in list_indices:
        result += (tup[idx-1],)
        
    return result

In [19]:
df = sqlContext.createDataFrame(final_rdd, schema = data_schema).coalesce(12).cache()

In [20]:
last = len(names)
list_indices = [i for i in range(2,last)]
L = df.rdd.count()
print(last)
print(L)

132
188318


In [21]:
structField_list_selected = [create_StructField(string) for i,string in enumerate(names) if i in list_indices]

In [22]:
mean = df.rdd.map(lambda x: (x[-1])).mean()
stdev = df.rdd.map(lambda x: (x[-1])).stdev()

In [23]:
import math
def transform_loss(x):
    return int(x-mean)

df_Perceptron = sqlContext.createDataFrame(df.rdd.map(lambda x: (transform_loss(x[-1]), Vectors.dense(keep_index(x)))), ["label", "features"])

In [24]:
df_Perceptron.select("label").show()

+-----+
|label|
+-----+
| -824|
|-1753|
|  -32|
|-2097|
| -273|
| 2105|
|-1905|
|  548|
| 7242|
| 3147|
| 3359|
| 2928|
|-1844|
|-1965|
|-2452|
|-1641|
| 3571|
| -378|
| 1129|
|  760|
+-----+
only showing top 20 rows



In [25]:
df_Perceptron.select("features").first()

Row(features=DenseVector([0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 1.0, 1.0, 3.0, 3.0, 1.0, 3.0, 2.0, 1.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 2.0, 4.0, 0.0, 2.0, 15.0, 1.0, 6.0, 0.0, 0.0, 8.0, 4.0, 6.0, 9.0, 6.0, 45.0, 28.0, 2.0, 19.0, 55.0, 0.0, 14.0, 269.0, 0.7263, 0.2459, 0.1876, 0.7896, 0.3101, 0.7184, 0.3351, 0.3026, 0.6714, 0.8351, 0.5697, 0.5946, 0.8225, 0.7148]))

<h1>DecisionTreeRegressor</h1>

In [26]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=32).fit(df_Perceptron)

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(df_Perceptron_train)

# Make predictions.
predictions = model.transform(df_Perceptron_test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

treeModel = model.stages[1]
# summary only
print(treeModel)

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
| -870.4509177027827|-3016|[0.0,0.0,0.0,0.0,...|
|   239.360745949251|-3000|[0.0,0.0,0.0,1.0,...|
|-1512.5543143664117|-2990|[1.0,0.0,0.0,0.0,...|
|-1512.5543143664117|-2982|[0.0,0.0,0.0,0.0,...|
|   239.360745949251|-2980|[0.0,0.0,0.0,0.0,...|
+-------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2319.11
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_48628d8fca3c94c39325) of depth 5 with 61 nodes


<h1>GeneralizedLinearRegression</h1>

In [27]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
# Load training data
(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)

# Fit the model
model = glr.fit(df_Perceptron_train)
predictions = model.transform(df_Perceptron_test)
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|-1384.3056890041448|-3028|[0.0,0.0,0.0,0.0,...|
|-1791.2807233646777|-3016|[0.0,0.0,0.0,0.0,...|
| 145.82262524537418|-2980|[0.0,0.0,0.0,0.0,...|
|-2083.7987068297894|-2975|[0.0,0.0,0.0,1.0,...|
|-1865.7978519117746|-2971|[1.0,0.0,0.0,0.0,...|
+-------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2046.89


<h1>RandomForestRegressor</h1>

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df_Perceptron)


# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(df_Perceptron_train)

# Make predictions.
predictions = model.transform(df_Perceptron_test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|-1020.4532384166043|-3028|[0.0,0.0,0.0,0.0,...|
|-1021.3137218427297|-3017|[1.0,0.0,0.0,0.0,...|
|-1020.4532384166043|-2982|[0.0,0.0,0.0,0.0,...|
| 2554.0547511405034|-2957|[0.0,0.0,0.0,1.0,...|
|-1253.5649723904537|-2937|[1.0,0.0,0.0,1.0,...|
+-------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2161.53
RandomForestRegressionModel (uid=rfr_2762b2dc24dc) with 20 trees


<h1>Gradient-boosted tree regression</h1>

In [ ]:
Fre

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df_Perceptron)

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(df_Perceptron_train)

# Make predictions.
predictions = model.transform(df_Perceptron_test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|-1340.1336595367884|-3016|[0.0,0.0,0.0,0.0,...|
|-1969.8272770670592|-2998|[1.0,0.0,0.0,0.0,...|
|-400.04707278890606|-2980|[0.0,0.0,0.0,0.0,...|
|-1405.6426707753062|-2976|[0.0,0.0,0.0,0.0,...|
|-1799.0712097941314|-2971|[1.0,0.0,0.0,0.0,...|
+-------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2165.9
GBTRegressionModel (uid=GBTRegressor_4769b99b79932a236350) with 10 trees


<h1>IsotonicRegression</h1>

In [ ]:
from pyspark.ml.regression import IsotonicRegression, IsotonicRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

# Trains an isotonic regression model.
model = IsotonicRegression().fit(df_Perceptron_train)
# Makes predictions.
predictions = model.transform(df_Perceptron_test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

<h1>One-vs-Rest classifier</h1>

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(df_Perceptron_train)

# score the model on test data.
predictions = ovrModel.transform(df_Perceptron_test)

# obtain evaluator.
rmse = MulticlassClassificationEvaluator(metricName="mae")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error : " + str(rmse))

In [ ]:
from pyspark.ml.feature import PCA


pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df_Perceptron_train)
result = model.transform(df_Perceptron_test).select("pcaFeatures")
result.show(truncate=False)
# $example off$

<h1>Decision tree classifier</h1>

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the data stored in LIBSVM format as a DataFrame.
(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_Perceptron)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df_Perceptron)


# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(df_Perceptron_train)

# Make predictions.
predictions = model.transform(df_Perceptron_test)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Test Error = %g " % (rmse))

treeModel = model.stages[2]
# summary only
print(treeModel)

<h1>LogisticRegression</h1>

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(df_Perceptron_train)

# score the model on test data.
predictions = ovrModel.transform(df_Perceptron_test)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="rmse")

# compute the classification error on test data.
rmse = evaluator.evaluate(predictions)
print("Test Error : " + str(rmse))

<h1>NaiveBayes</h1>

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

(df_Perceptron_train,df_Perceptron_test) = df_Perceptron.randomSplit([0.7, 0.3])
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(df_Perceptron_train)
# compute accuracy on the test set
result = model.transform(df_Perceptron_test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="rmse")
print("rmse: " + str(evaluator.evaluate(predictionAndLabels)))